In [299]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_diabetes

In [300]:
data = load_diabetes()
X = data.data
y = data.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
y[0]

151.0

In [301]:
X_train_num = np.dot(X_train,1000).astype(int)
X_test_num = np.dot(X_test,1000).astype(int)
y_train_num = y_train
y_test_num = y_test

In [302]:
print(X_train_num)

[[ 70  50  12 ...  34  27  -1]
 [ -9  50 -18 ...  71   0  19]
 [  5 -44  49 ...  -2  17 -13]
 ...
 [ 30 -44 -20 ... -39 -10  -1]
 [-12 -44 -23 ...  -2 -38 -38]
 [-92 -44  28 ... -39  -5  -1]]


In [303]:
X_train_den = np.full(X_train.shape, 1000) 
X_test_den = np.full(X_test.shape, 1000) 
y_train_den = np.ones(len(y_train))
y_test_den = np.ones(len(y_test))

In [304]:
print(X_train_den)

[[1000 1000 1000 ... 1000 1000 1000]
 [1000 1000 1000 ... 1000 1000 1000]
 [1000 1000 1000 ... 1000 1000 1000]
 ...
 [1000 1000 1000 ... 1000 1000 1000]
 [1000 1000 1000 ... 1000 1000 1000]
 [1000 1000 1000 ... 1000 1000 1000]]


In [305]:
#condensed function multiply and add matrix with numerator and denominator one row at a time
# Python Program to find the L.C.M. of two input number
from math import gcd
def compute_lcm(x, y):
   # choose the greater number 
    if (x > y):
        greater = x
    else:
        greater = y
    while(True):
        if(x==0):
            lcm = y
            break
        elif(y==0):
            lcm = x
            break
        elif((greater % x == 0) and (greater % y == 0)):
            lcm = greater
            break
        greater += 1
    return lcm
 
#Subtracting 2 numbers of the form a/b -c/d
def subtraction(a,b,c,d):
    den = compute_lcm(b,d)
    if b ==0:
        num = c ; den = d
    elif d==0:
        num= a ; den = b
    else:    
        num = ((den/b)*a) - ((den/d)*c)
    return num,den

#adding array of elements of the form x/y
def add_array(a_num,a_den):
    #will work for an int array of any length
    lcm = 1;sum1 =0
    for i in a_den:
        lcm = int(lcm*i/gcd(lcm, i))
    print("lcm is ",lcm)
    for i in range(len(a_num)):
        sum1 = sum1 + (a_num[i]*(a_den[i]/lcm))
#         print(a_num[i],a_den[i],lcm)      
#     print("sum is ", sum1)
    return sum1,lcm

#adding two numbers of the form a/b + c/d
def addition(a,b,c,d):
    den = compute_lcm(b,d)
    if b ==0:
        num = c ; den = d
    elif d==0:
        num= a ; den = b
    else:    
        num = ((den/b)*a) + ((den/d)*c)
    return int(num),int(den)

#multiplying two arrays of fraction and adding them  for row by column multiplicatino
def multiply(a_num,a_den,b_num, b_den):
#     print(type(a_num))
    z_num = np.zeros(len(a_num)).astype(int)
    z_den = np.zeros(len(a_den)).astype(int)
    print(a_num,a_den)                                   
    for i in range(len(a_num)):
        z_num[i]=(a_num[i]*b_num[i])     
        z_den[i]=(a_den[i]*b_den[i])
    
    x = z_num.tolist()
    y = z_den.tolist()
    print("Printing z_num and z_den", x , y )
    num,den = add_array(x,y)
    print("multiplication results is",num,den) 
    return num,den
    

In [306]:
#claculate y_hat =wx+b | x is in the form a single array and w is in the form of array too
# need toc hange w into wnum and wden
def y_hat(w_num,w_den,x_num,x_den,b):
    y_hat_num = np.zeros(x_num.shape[0]).astype(int)
    y_hat_den = np.zeros(x_den.shape[0]).astype(int)
    #multiplying w and x
    for i in range(x_num.shape[0]):
#         print("iteration number ",i)
        x_n = x_num[i] 
        x_d = x_den[i] 
#         print(x_n,x_d,w_num,w_den)
        temp_n, temp_d = multiply(x_n,x_d,w_num,w_den)
#         print("printing temporary values",temp_n,temp_d)
        y_hat_num[i],y_hat_den[i] = addition( temp_n, temp_d,b[0],b[1])          
#     print("y_hat values for this iterationsa are",y_hat_num,y_hat_den)
    return y_hat_num,y_hat_den

In [307]:
#calculating mean square values
def mse(y_hat_num,y_hat_den,y_num,y_den,n):
    num=0;den =0
    for i in range(len(y_hat_num)):
        num1,den1=subtraction(y_num[i],y_den[i],y_hat_num[i],y_hat_den[i])
        num1 = num1*num1
        den1 = den1*den1
        num,den = addition(num,den,num1,den1)
    den = den * n
#     print(num,den)
    return num,den

In [308]:
#calculating subtraction of Y_hat Y
def y_hat_substraction(y_hat_num,y_hat_den,y_num,y_den):
    y_diff =np.zeros((y_hat_num.shape[0],2))
    for i in range(len(y_hat_num)):
        y_diff[i][0],y_diff[i][1] = subtraction(y_hat_num[i],y_hat_den[i], y_num[i], y_den[i])
    return y_diff



In [309]:
#getting delta w with respect 
#need change - can call the subtraction of y_hat from y
def delta_w(y_hat_diff,x_num, x_den, n):
    del_w_num = np.zeros(x_num.shape[1])
    del_w_den = np.zeros(x_num.shape[1])
    
    x_num = x_num.T
    x_den = x_den.T
    y_num = y_hat_diff[:,0]
    y_den = y_hat_diff[:,1]
    
#     print("getting delta w num and den ")
    for i in range(len(del_w_num)):
        print("printing X",len(x_num[i]))
        print("printing X",len(x_den[i]))
        print("printing y diff",len( y_num))
        print("printing y diff", len(y_den))
        del_w_num[i],del_w_den[i] = multiply(x_num[i],x_den[i],y_num,y_den)
        del_w_num[i] = del_w_num[i]*2
        del_w_den[i] = del_w_den[i]*n
    print("delta w num and den are",del_w_num,del_w_den)
    return del_w_num,del_w_den

    
    

In [310]:
#getting delta b 
def delta_b(y_hat_diff):

#     print(type(y_hat_diff))
    yh_num = np.array(y_hat_diff[:,0]).astype(int)
    yh_den = np.array(y_hat_diff[:,1]).astype(int)
    print(type(yh_num[0]))
    print(type(yh_den[0]))    
    print("printing yh_num",yh_num)
    print("printing yh_den",yh_den)
    num,den = add_array(yh_num, yh_den)
    num = num*2
    den = den*len(yh_num)
#     print("printing delta_b", num,den)
    return num,den

In [311]:
# get new weights and biases
def get_new_params(lr, dw_num,dw_den,db_num,db_den,w_num,w_den,b_num,b_den):
    
    for i in range(len(dw_num)):
        dw_num[i]= dw_num[i]*lr[0]
        dw_den[i]= dw_den[i]*lr[1]
   
    btemp_num = lr[0] * db_num
    btemp_den = lr[1] * db_den 

    for i in range(len(dw_num)):
        dw_num[i],dw_den[i] = subtraction(w_num[i],w_den[i],dw_num[i],dw_den[i]) 
    
    db_num,db_den = subtraction(b_num,b_den,btemp_num,btemp_den)
    print("printing new weight parameters", dw_num,dw_den)
    print("printing new bias parameters",db_num,db_den)
    return  dw_num,dw_den,db_num,db_den

In [312]:
#normalize weights and bias
def normalize_weights_bias(wnum,wden, bnum,bden):
    wn_num = np.zeros(10) 
    wn_den = np.zeros(10)
    for i in range (len(wf_num)):
        wn_num[i] = int((wnum[i]/wden[i]) * 100)
        wn_den[i] = 100
    bnum = int(bnum/bden * 100)
    bden = 100
    
    print("Normalizing weights and biases",wn_num,wn_den, bnum, bden)
    return wn_num, wn_den,bnum, bden

In [313]:
wnum = np.ones(10)
wden = np.ones(10)
b =[1,1]
bnum =1
bden =1
lr = [1,100]

In [314]:
#getting y_hat = wx+b and , this matches with my main code
yh_num,yh_den = y_hat(wnum,wden,X_train_num, X_train_den,b)
print(yh_hum,yh_den)


[ 70  50  12  56  34  49 -39  34  27  -1] [1000 1000 1000 1000 1000 1000 1000 1000 1000 1000]
Printing z_num and z_den [70, 50, 12, 56, 34, 49, -39, 34, 27, -1] [1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000]
lcm is  1000
multiplication results is 292.0 1000
[ -9  50 -18 -33 -20  12 -72  71   0  19] [1000 1000 1000 1000 1000 1000 1000 1000 1000 1000]
Printing z_num and z_den [-9, 50, -18, -33, -20, 12, -72, 71, 0, 19] [1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000]
lcm is  1000
multiplication results is 0.0 1000
[  5 -44  49  97 -15 -16  -6  -2  17 -13] [1000 1000 1000 1000 1000 1000 1000 1000 1000 1000]
Printing z_num and z_den [5, -44, 49, 97, -15, -16, -6, -2, 17, -13] [1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000]
lcm is  1000
multiplication results is 72.0 1000
[ -27  -44  -35  -29  -56  -58   30  -39  -49 -129] [1000 1000 1000 1000 1000 1000 1000 1000 1000 1000]
Printing z_num and z_den [-27, -44, -35, -29, -56, -58, 30, -39, -49, -129] [10

lcm is  1000
multiplication results is -340.0 1000
[ 56  50  21  56  -7  18 -32  -2 -23  23] [1000 1000 1000 1000 1000 1000 1000 1000 1000 1000]
Printing z_num and z_den [56, 50, 21, 56, -7, 18, -32, -2, -23, 23] [1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000]
lcm is  1000
multiplication results is 160.0 1000
[-85 -44  20 -26   5  19  -2  -2 -23   3] [1000 1000 1000 1000 1000 1000 1000 1000 1000 1000]
Printing z_num and z_den [-85, -44, 20, -26, 5, 19, -2, -2, -23, 3] [1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000]
lcm is  1000
multiplication results is -135.0 1000
[-23 -44 -15 -12  20  41 -43  34  14  -9] [1000 1000 1000 1000 1000 1000 1000 1000 1000 1000]
Printing z_num and z_den [-23, -44, -15, -12, 20, 41, -43, 34, 14, -9] [1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000]
lcm is  1000
multiplication results is -37.0 1000
[ 38  50   8  42 -42 -21 -39  -2 -18   7] [1000 1000 1000 1000 1000 1000 1000 1000 1000 1000]
Printing z_num and z_den [38, 50

multiplication results is -145.0 1000
[  9  50  -1  21 -38 -24  -6 -39 -39 -13] [1000 1000 1000 1000 1000 1000 1000 1000 1000 1000]
Printing z_num and z_den [9, 50, -1, 21, -38, -24, -6, -39, -39, -13] [1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000]
lcm is  1000
multiplication results is -80.0 1000
[  1  50  11 -19 -16  -3 -47  34  24  23] [1000 1000 1000 1000 1000 1000 1000 1000 1000 1000]
Printing z_num and z_den [1, 50, 11, -19, -16, -3, -47, 34, 24, 23] [1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000]
lcm is  1000
multiplication results is 58.0 1000
[ 19 -44   4  45 -18 -54  63 -39  28  61] [1000 1000 1000 1000 1000 1000 1000 1000 1000 1000]
Printing z_num and z_den [19, -44, 4, 45, -18, -54, 63, -39, 28, 61] [1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000]
lcm is  1000
multiplication results is 65.0 1000
[ 16  50 -46  11 -33 -16 -10  -2 -43 -42] [1000 1000 1000 1000 1000 1000 1000 1000 1000 1000]
Printing z_num and z_den [16, 50, -46, 11, -33, 

NameError: name 'yh_hum' is not defined

In [ ]:
#getting MSE, MSE matches with my main code
mse_num,mse_den= mse(yh_num, yh_den, y_train_num,y_train_den,len(yh_num))

print("mse is ", mse_num, mse_den)

In [315]:
y_diff = y_hat_substraction(yh_num,yh_den,y_train_num,y_train_den)
print("difference in y and yhat is ",y_diff)



difference in y and yhat is  [[-142708.    1000.]
 [-149000.    1000.]
 [-278928.    1000.]
 [-124436.    1000.]
 [ -58400.    1000.]
 [ -64444.    1000.]
 [-279968.    1000.]
 [-275656.    1000.]
 [-166089.    1000.]
 [ -89265.    1000.]
 [ -71167.    1000.]
 [-176830.    1000.]
 [ -87388.    1000.]
 [-268787.    1000.]
 [-100355.    1000.]
 [-195815.    1000.]
 [ -96514.    1000.]
 [ -52012.    1000.]
 [ -69915.    1000.]
 [-260718.    1000.]
 [ -51170.    1000.]
 [-100978.    1000.]
 [-164843.    1000.]
 [-187770.    1000.]
 [-171702.    1000.]
 [-218823.    1000.]
 [-205133.    1000.]
 [ -96249.    1000.]
 [ -59114.    1000.]
 [ -60411.    1000.]
 [-241003.    1000.]
 [-120135.    1000.]
 [-127228.    1000.]
 [-103172.    1000.]
 [-263841.    1000.]
 [-131023.    1000.]
 [-281876.    1000.]
 [-172974.    1000.]
 [-128290.    1000.]
 [-255923.    1000.]
 [-136536.    1000.]
 [ -61617.    1000.]
 [ -92221.    1000.]
 [-230935.    1000.]
 [-206891.    1000.]
 [-259691.    1000.]
 [-17

In [316]:
# get delta_W
delw_num,delw_den = delta_w(y_diff,X_train_num,X_train_den, len(yh_num))
print("printing delta_w", delw_num,delw_den)


printing X 353
printing X 353
printing y diff 353
printing y diff 353
[  70   -9    5  -27  -23    1   16    9   -9  -78   56   41    1   30
  -63   67  -92   27   41   23   16   67   -5   63    9   16  -89  -38
  -74  -70   12   -5  -34   41    9   41   67   -5   23   34 -107   63
   16   19   30   74  -56   23    1   -5   74  -85    5    1  -74  -27
   30    9   -1    9   38   27  -41    9  -34   23  -38  -38   56   41
   12  -23  -41   23   16   41   16  -23  -20   67  -27   -5  -70   81
   12   23   70  -27  -12   67   63   19  110   -9  -45   52   -9  -12
   23   48   16  -96  -52   59   12   -1  -34   70   81   38  -52   48
   30   -1  -67  -45   -1   16  -81  -56   85  -20  -52   16   38    5
  -16   56  -30    5   19  -34  -27   12   38   45   -9   30   48   81
   81   45   41   45   19  -16   -5   34   41  -41   -5  -30  -63  -52
  -99  -52  -70   52  -89    9   48  -16  -74  -60   45  -74  -23  -74
   56  -85  -23   38   38   59  -78   88   67   16   45  -92   48   78
   12  

In [317]:
# get delta_b
delb_num, delb_den = delta_b(y_diff)
print("printing delta_b", delb_num,delb_den)



<class 'numpy.int32'>
<class 'numpy.int32'>
printing yh_num [-142708 -149000 -278928 -124436  -58400  -64444 -279968 -275656 -166089
  -89265  -71167 -176830  -87388 -268787 -100355 -195815  -96514  -52012
  -69915 -260718  -51170 -100978 -164843 -187770 -171702 -218823 -205133
  -96249  -59114  -60411 -241003 -120135 -127228 -103172 -263841 -131023
 -281876 -172974 -128290 -255923 -136536  -61617  -92221 -230935 -206891
 -259691 -177857 -256749 -260872  -50185 -235768  -70402 -137515 -266523
  -68343 -315753 -247693 -153006 -190756 -114768  -79804 -120786 -258052
 -189967 -290667  -54103 -105953 -209336  -89970 -251706  -83758 -251096
  -58337  -77248 -199090  -76934 -243964 -173997  -41401 -126153  -52028
  -92935 -103350 -198133 -263951 -279971 -246543 -255884 -213774 -301823
 -168704  -58055 -275774 -207864 -137260 -197109 -123074  -95262 -287036
 -223830 -264042 -100593  -54175 -196770  -49945 -250477  -63282 -218660
 -129521 -210699 -141341 -101819 -153720 -119626  -85262 -110149

In [318]:
#get new parameters
wf_num,wf_den,bf_num,bf_den = get_new_params(lr, delw_num,delw_den,delb_num,delb_den,wnum,wden,bnum,bden)



printing new weight parameters [3.59394489e+10 3.53375973e+10 3.70446325e+10 3.66004778e+10
 3.57590156e+10 3.56139281e+10 3.42229814e+10 3.64692167e+10
 3.68681834e+10 3.65277629e+10] [3.53e+10 3.53e+10 3.53e+10 3.53e+10 3.53e+10 3.53e+10 3.53e+10 3.53e+10
 3.53e+10 3.53e+10]
printing new bias parameters 143127636.0 35300000


In [319]:
#normalize weights and bias parameters
wnum,wden, bnum, bden = normalize_weights_bias(wf_num,wf_den,bf_num,bf_den)
b[0] = bnum
b[1] = bden

Normalizing weights and biases [101. 100. 104. 103. 101. 100.  96. 103. 104. 103.] [100. 100. 100. 100. 100. 100. 100. 100. 100. 100.] 405 100


In [320]:
#getting y_hat = wx+b and , this matches with my main code
yh_num,yh_den = y_hat(wnum,wden,X_train_num, X_train_den,b)
print("printing y_hat results",yh_num, yh_den)

[ 70  50  12  56  34  49 -39  34  27  -1] [1000 1000 1000 1000 1000 1000 1000 1000 1000 1000]
Printing z_num and z_den [7070, 5000, 1248, 5768, 3434, 4900, -3744, 3502, 2808, -103] [100000, 100000, 100000, 100000, 100000, 100000, 100000, 100000, 100000, 100000]
lcm is  100000
multiplication results is 29883.0 100000
[ -9  50 -18 -33 -20  12 -72  71   0  19] [1000 1000 1000 1000 1000 1000 1000 1000 1000 1000]
Printing z_num and z_den [-909, 5000, -1872, -3399, -2020, 1200, -6912, 7313, 0, 1957] [100000, 100000, 100000, 100000, 100000, 100000, 100000, 100000, 100000, 100000]
lcm is  100000
multiplication results is 358.0 100000
[  5 -44  49  97 -15 -16  -6  -2  17 -13] [1000 1000 1000 1000 1000 1000 1000 1000 1000 1000]
Printing z_num and z_den [505, -4400, 5096, 9991, -1515, -1600, -576, -206, 1768, -1339] [100000, 100000, 100000, 100000, 100000, 100000, 100000, 100000, 100000, 100000]
lcm is  100000
multiplication results is 7724.0 100000
[ -27  -44  -35  -29  -56  -58   30  -39  -49 -

[ 88 -44   6  25  30   8  63 -39   9  32] [1000 1000 1000 1000 1000 1000 1000 1000 1000 1000]
Printing z_num and z_den [8888, -4400, 624, 2575, 3030, 800, 6048, -4017, 936, 3296] [100000, 100000, 100000, 100000, 100000, 100000, 100000, 100000, 100000, 100000]
lcm is  100000
multiplication results is 17780.0 100000
[ 67 -44  36 -50 -23 -34  48 -39 -25 -38] [1000 1000 1000 1000 1000 1000 1000 1000 1000 1000]
Printing z_num and z_den [6767, -4400, 3744, -5150, -2323, -3400, 4608, -4017, -2600, -3914] [100000, 100000, 100000, 100000, 100000, 100000, 100000, 100000, 100000, 100000]
lcm is  100000
multiplication results is -10685.0 100000
[ 16 -44 -24  35  -7  -3 -13  -2  15  15] [1000 1000 1000 1000 1000 1000 1000 1000 1000 1000]
Printing z_num and z_den [1616, -4400, -2496, 3605, -707, -300, -1248, -206, 1560, 1545] [100000, 100000, 100000, 100000, 100000, 100000, 100000, 100000, 100000, 100000]
lcm is  100000
multiplication results is -1031.0 100000
[ 45 -44 -19  21  27 -13 100 -39  17 -1

Printing z_num and z_den [505, 5000, 3120, 8549, -3737, -4700, 1440, -4017, 832, 1545] [100000, 100000, 100000, 100000, 100000, 100000, 100000, 100000, 100000, 100000]
lcm is  100000
multiplication results is 8537.0 100000
[ 30 -44 -20  -5  -4 -29  78 -39 -10  -1] [1000 1000 1000 1000 1000 1000 1000 1000 1000 1000]
Printing z_num and z_den [3030, -4400, -2080, -515, -404, -2900, 7488, -4017, -1040, -103] [100000, 100000, 100000, 100000, 100000, 100000, 100000, 100000, 100000, 100000]
lcm is  100000
multiplication results is -4941.0 100000
[-12 -44 -23 -40 -16   4 -17  -2 -38 -38] [1000 1000 1000 1000 1000 1000 1000 1000 1000 1000]
Printing z_num and z_den [-1212, -4400, -2392, -4120, -1616, 400, -1632, -206, -3952, -3914] [100000, 100000, 100000, 100000, 100000, 100000, 100000, 100000, 100000, 100000]
lcm is  100000
multiplication results is -23044.0 100000
[-92 -44  28 -15  36  24  56 -39  -5  -1] [1000 1000 1000 1000 1000 1000 1000 1000 1000 1000]
Printing z_num and z_den [-9292, -44

In [321]:
#getting MSE, MSE matches with my main code
mse_num,mse_den= mse(yh_num, yh_den, y_train_num,y_train_den,len(yh_num))
print(mse_num, mse_den)

100466191009333664 497753089024


<ipython-input-307-d1d7b0152d37>:7: RuntimeWarning: overflow encountered in long_scalars
  den1 = den1*den1


In [322]:
y_diff = y_hat_substraction(yh_num,yh_den,y_train_num,y_train_den)
y_diff

array([[-13965117.,    100000.],
       [-14594642.,    100000.],
       [-27587276.,    100000.],
       [-12139730.,    100000.],
       [ -5536302.,    100000.],
       [ -6140869.,    100000.],
       [-27691464.,    100000.],
       [-27259200.,    100000.],
       [-16303627.,    100000.],
       [ -8622515.,    100000.],
       [ -6812602.,    100000.],
       [-17377460.,    100000.],
       [ -8434215.,    100000.],
       [-26572367.,    100000.],
       [ -9731276.,    100000.],
       [-19276305.,    100000.],
       [ -9347621.,    100000.],
       [ -4896400.,    100000.],
       [ -6686503.,    100000.],
       [-25766504.,    100000.],
       [ -4812092.,    100000.],
       [ -9792718.,    100000.],
       [-16179090.,    100000.],
       [-18471296.,    100000.],
       [-16864314.,    100000.],
       [-21576578.,    100000.],
       [-20208222.,    100000.],
       [ -9320140.,    100000.],
       [ -5606558.,    100000.],
       [ -5737114.,    100000.],
       [-2

In [323]:
# get delta_W
delw_num,delw_den = delta_w(y_diff,X_train_num,X_train_den, len(yh_num))

printing X 353
printing X 353
printing y diff 353
printing y diff 353
[  70   -9    5  -27  -23    1   16    9   -9  -78   56   41    1   30
  -63   67  -92   27   41   23   16   67   -5   63    9   16  -89  -38
  -74  -70   12   -5  -34   41    9   41   67   -5   23   34 -107   63
   16   19   30   74  -56   23    1   -5   74  -85    5    1  -74  -27
   30    9   -1    9   38   27  -41    9  -34   23  -38  -38   56   41
   12  -23  -41   23   16   41   16  -23  -20   67  -27   -5  -70   81
   12   23   70  -27  -12   67   63   19  110   -9  -45   52   -9  -12
   23   48   16  -96  -52   59   12   -1  -34   70   81   38  -52   48
   30   -1  -67  -45   -1   16  -81  -56   85  -20  -52   16   38    5
  -16   56  -30    5   19  -34  -27   12   38   45   -9   30   48   81
   81   45   41   45   19  -16   -5   34   41  -41   -5  -30  -63  -52
  -99  -52  -70   52  -89    9   48  -16  -74  -60   45  -74  -23  -74
   56  -85  -23   38   38   59  -78   88   67   16   45  -92   48   78
   12  

OverflowError: Python int too large to convert to C long

In [ ]:
# get delta_b
delb_num, delb_den = delta_b(y_diff)


In [ ]:

#get new parameters
wf_num,wf_den,bf_num,bf_den = get_new_params(lr, delw_num,delw_den,delb_num,delb_den,wnum,wden,bnum,bden)



In [ ]:
#normalize weights and bias parameters
wnum,wden, bnum, bden = normalize_weights_bias(wf_num,wf_den,bf_num,bf_den)